In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from sklearn.preprocessing import StandardScaler

In [ ]:
'''
CO(mg/m^3)_lag_1: MAE: 0.3926831388089995
CO(mg/m^3)_lag_2: MAE: 0.4968563914970162
CO(mg/m^3)_lag_3: MAE: 0.5501705725838607
CO(mg/m^3)_lag_4: MAE: 0.5715380364944085
CO(mg/m^3)_lag_5: MAE: 0.628628765641844
CO(mg/m^3)_lag_6: MAE: 0.6424215457554477
Mean MAE: 0.5470497417969294
'''